# <font color='blue'>Data Science Academy - Deep Learning II</font>

# Regularização

# L1e L2 no TensorFlow

Adicionando Regularização L1 e L2 do Modelo de Rede Neural com TensorFlow: http://tensorlayer.readthedocs.io/en/stable/modules/cost.html.

In [1]:
########################################
# TensorFlow com L1/L2 Para Regressão
########################################
%matplotlib inline
from matplotlib.pyplot import figure, show
import tensorflow as tf
from sklearn.model_selection import train_test_split
import pandas as pd
import os
import numpy as np
from sklearn import metrics
from scipy.stats import zscore
from keras.callbacks import EarlyStopping
from keras.layers import Dense, Dropout
from keras import regularizers
from keras.models import Sequential

Using TensorFlow backend.


In [2]:
# Carregando os Dados e difinindo o nível de log
path = "./data/"

filename_read = os.path.join(path,"auto-mpg.csv")
df = pd.read_csv(filename_read,na_values=['NA','?'])

# Define o nível de saída TensorFlow desejado para este exemplo
tf.logging.set_verbosity(tf.logging.ERROR)

In [3]:
# Função Auxiliares

# Converte todos os valores faltantes na coluna especificada para a mediana
def missing_median(df, name):
    med = df[name].median()
    df[name] = df[name].fillna(med)


# Converte um dataframe Pandas para as entradas x, y que o TensorFlow precisa
def to_xy(df, target):
    result = []
    for x in df.columns:
        if x != target:
            result.append(x)
    # Descobre o tipo da coluna de destino. 
    target_type = df[target].dtypes
    target_type = target_type[0] if hasattr(target_type, '__iter__') else target_type
    # Encoding para int. TensorFlow gosta de 32 bits.
    if target_type in (np.int64, np.int32):
        # Classificação
        dummies = pd.get_dummies(df[target])
        return df.as_matrix(result).astype(np.float32), dummies.as_matrix().astype(np.float32)
    else:
        # Regressão
        return df.as_matrix(result).astype(np.float32), df.as_matrix([target]).astype(np.float32)

In [4]:
# Pré-processaento nos dadod
df.drop('name', 1, inplace = True)
missing_median(df, 'horsepower')
x,y = to_xy(df,"mpg")

In [5]:
print(x)

[[   8.          307.          130.         ...,   12.           70.            1.        ]
 [   8.          350.          165.         ...,   11.5          70.            1.        ]
 [   8.          318.          150.         ...,   11.           70.            1.        ]
 ..., 
 [   4.          135.           84.         ...,   11.60000038   82.            1.        ]
 [   4.          120.           79.         ...,   18.60000038   82.            1.        ]
 [   4.          119.           82.         ...,   19.39999962   82.            1.        ]]


In [6]:
print(y)

[[ 18.        ]
 [ 15.        ]
 [ 18.        ]
 [ 16.        ]
 [ 17.        ]
 [ 15.        ]
 [ 14.        ]
 [ 14.        ]
 [ 14.        ]
 [ 15.        ]
 [ 15.        ]
 [ 14.        ]
 [ 15.        ]
 [ 14.        ]
 [ 24.        ]
 [ 22.        ]
 [ 18.        ]
 [ 21.        ]
 [ 27.        ]
 [ 26.        ]
 [ 25.        ]
 [ 24.        ]
 [ 25.        ]
 [ 26.        ]
 [ 21.        ]
 [ 10.        ]
 [ 10.        ]
 [ 11.        ]
 [  9.        ]
 [ 27.        ]
 [ 28.        ]
 [ 25.        ]
 [ 25.        ]
 [ 19.        ]
 [ 16.        ]
 [ 17.        ]
 [ 19.        ]
 [ 18.        ]
 [ 14.        ]
 [ 14.        ]
 [ 14.        ]
 [ 14.        ]
 [ 12.        ]
 [ 13.        ]
 [ 13.        ]
 [ 18.        ]
 [ 22.        ]
 [ 19.        ]
 [ 18.        ]
 [ 23.        ]
 [ 28.        ]
 [ 30.        ]
 [ 30.        ]
 [ 31.        ]
 [ 35.        ]
 [ 27.        ]
 [ 26.        ]
 [ 24.        ]
 [ 25.        ]
 [ 23.        ]
 [ 20.        ]
 [ 21.        ]
 [ 13.  

In [7]:
# Split em treino e teste
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.25, random_state = 45)

In [8]:
# Criação do Modelo
# https://keras.io/layers/core/
model = Sequential()
model.add(Dense(50, input_dim = x.shape[1], kernel_initializer = 'normal', activation = 'relu'))
model.add(Dense(25, input_dim = x.shape[1], kernel_initializer = 'normal', activation = 'relu'))

model.add(Dense(10, input_dim = 64, 
                kernel_regularizer = regularizers.l2(0.01), 
                activity_regularizer = regularizers.l1(0.01),
                activation = 'relu'))

model.add(Dense(1, kernel_initializer = 'normal'))

# Compilação
model.compile(loss = 'mean_squared_error', optimizer = 'adam')

# Early Stopping
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=1, mode='auto')

# Fit
model.fit(x,y,validation_data=(x_test,y_test),callbacks=[monitor],verbose=0,epochs=1000)

# Previsões
pred = model.predict(x_test)

Epoch 00119: early stopping


In [9]:
# Measure RMSE error.  RMSE is common for regression.
score = np.sqrt(metrics.mean_squared_error(pred,y_test))
print("Final score (RMSE): {}".format(score))

Final score (RMSE): 2.9664759635925293


# Fim